In [1]:
import requests
import json
from bs4 import BeautifulSoup
import os
import re
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from pathlib import Path  # Python 3.6+ only`
env_path = Path('.') / '.env'
load_dotenv(dotenv_path=env_path)

True

In [2]:
#### Sample playlist
kittinan_links = 'https://gist.githubusercontent.com/kittinan/36b9977d9273755f9ca953cae31419d3/raw/50383e756bc34cfede4c58018f909805ecd1cfbc/my-playlist.json'
dct_kittinan_playlist = requests.get(kittinan_links).json()

In [3]:
### scrape artist siamzone
base_url = 'https://www.siamzone.com/music/thailyric/index.php?mode=allartist'
i = 1
soup = BeautifulSoup(requests.get(f'{base_url}&p={i}').text)
lst_pages = ['1'] + [tag.get_text()
                    for tag in soup.find_all('a', attrs={'class':'uk-link', 'href':re.compile(r'\/music\/thailyric\/index.php')})
                    if len(tag.get_text())>0]

lst_soup = [BeautifulSoup(requests.get(f'{base_url}&p={page}').text) for page in lst_pages]
crawl_now = [artist for artist in [tag.get_text(strip=True).partition(' ')[-1]
                                   for soup in lst_soup
                                   for tag in soup.find_all('a', attrs={'class':'uk-link'})]
             if len(artist)>0]

/usr/lib/python3/dist-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [4]:
#### Spotify Authentication
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=os.getenv('SPOTIFY_CLIENT_ID'),
                                                           client_secret=os.getenv('SPOTIFY_SECRET')))

In [5]:

playlists = sp.user_playlists('spotify')
lst_playlist = []
while playlists:
    lst_playlist += [playlist for playlist in playlists['items']]
    playlists = sp.next(playlists) if playlists['next'] else None
    
df_spotify_playlist = pd.DataFrame(lst_playlist)

In [ ]:
#### Get siamzone
lst_df = []
for artist in crawl_now:
    results = sp.search(q=artist, limit=50)
    lst_df.append(pd.DataFrame(results['tracks']['items']))
    df_spotify_query = pd.concat(lst_df, ignore_index=True)